## Linked List

In [1]:
help(isinstance)

Help on built-in function isinstance in module builtins:

isinstance(obj, class_or_tuple, /)
    Return whether an object is an instance of a class or of a subclass thereof.
    
    A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
    check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
    or ...`` etc.



In [34]:
class Link:
    empty=()
    
    def __init__(self,first,rest=empty):
        assert rest is Link.empty or isinstance(rest,Link)
        self.first = first
        self.rest = rest
        
    def __getitem__(self,i):
        if i==0:
            return self.first
        else:
            return self.rest[i-1]
        
    def __len__(self):
        return 1+len(self.rest)
    
    def __repr__(self):
        if self.rest:
            rest_str = ', '+ repr(self.rest)
        else:
            rest_str = ''
        return 'Link({0}{1})'.format(self.first, rest_str)
    
    @property
    def second(self):
        return self.rest.first
    
    @second.setter
    def second(self,value):
        self.rest.first = value
#感觉像__repr__这种函数应该是overwrite了python中原有的一些函数，比如我们这时候就饿
#可以print一个Linked List之类了

In [31]:
s = Link(3,Link(4,Link(5)))

In [22]:
s.first

3

In [23]:
s.rest.first

4

In [24]:
s.rest.rest.first

5

In [25]:
s.rest.rest.rest is Link.empty

True

In [26]:
Link(8,s.rest)

Link(8, Link(4, Link(5)))

In [27]:
s

Link(3, Link(4, Link(5)))

In [29]:
s.second() #这是没有加 @property的时候，必须要加()才能call

4

In [32]:
s.second 
#这是加了@property之后， as if "second" is just a regular instance attribute of s

4

In [33]:
s.rest.second

5

## Tree Class

In [41]:
class Tree:
    """A Tree is a label and a list of branches"""
    def __init__(self,label,branches=[]):
        self.label = label
        for b in branches:
            assert isinstance(b,Tree)
        self.branches = list(branches) # make a copy of the list
    
    def __repr__(self):
        if self.branches:
            branch_str = ', '+repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(self.label,branch_str)
    
    def __str__(self):
        return '\n'.join(self.indented())
    
    def indented(self):
        lines = []
        for b in self.branches:
            for line in b.indented():
                lines.append('  '+line)
        return [str(self.label)] + lines
    
    def is_leaf(self):
        return not self.branches

In [2]:
print('\n'.join(['a','b']))

a
b


In [3]:
def memo(f):
    cache={}
    def memorized(n):
        if n not in cache:
            cache[n] = f(n)
        return cache[n]
    return memorized

In [4]:
@memo
def fib_tree(n):
    if n==0 or n==1:
        return Tree(n)
    else:
        left=fib_tree(n-2)
        right=fib_tree(n-1)
        fib_n=left.label+right.label
        return Tree(fib_n,[left,right])

In [5]:
print(fib_tree(5))

5
  2
    1
    1
      0
      1
  3
    1
      0
      1
    2
      1
      1
        0
        1


In [6]:
fib_tree(5)

Tree(5, [Tree(2, [Tree(1), Tree(1, [Tree(0), Tree(1)])]), Tree(3, [Tree(1, [Tree(0), Tree(1)]), Tree(2, [Tree(1), Tree(1, [Tree(0), Tree(1)])])])])

In [7]:
def leaves(tree):
    if tree.is_leaf():
        return [tree.label]
    else:
        s=[]
        for b in tree.branches:
            s.extend(leaves(b))
        return s

In [8]:
leaves(fib_tree(6))

[0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]

In [9]:
def prune_branches(tree,seen):
    tree.branches = [b for b in tree.branches if b not in seen]
    seen.append(tree)
    for b in tree.branches:
        prune_branches(b,seen)

In [10]:
t = fib_tree(5)

In [11]:
t

Tree(5, [Tree(2, [Tree(1), Tree(1, [Tree(0), Tree(1)])]), Tree(3, [Tree(1, [Tree(0), Tree(1)]), Tree(2, [Tree(1), Tree(1, [Tree(0), Tree(1)])])])])

In [12]:
print(t)

5
  2
    1
    1
      0
      1
  3
    1
      0
      1
    2
      1
      1
        0
        1


In [13]:
prune_branches(t,[])

In [14]:
print(t)

5
  2
    1
    1
      0
  3


In [26]:
def eval_tree(t):
    def prod(lst):
        p = 1
        for x in lst:
            p*=x
        return p
    
    if t.is_leaf():
        return t.label
    else:
        if t.label == '+':
            return sum([eval_tree(b) for b in t.branches])
        else:
            return prod([eval_tree(b) for b in t.branches])

In [27]:
eval_tree(Tree(1))

1

In [28]:
expr = Tree('*',[Tree(2),Tree(3)])

In [29]:
eval_tree(expr)

6

In [30]:
eval_tree(Tree('+',[expr,Tree(4),Tree(5)]))

15

In [34]:
def f():
    print(1)
    return 'Yes'

In [35]:
f()[2:]

1


's'

## 0309_note Binary Tree

In [176]:
class BTree(Tree):
    """A tree with exactly two branches, which may be empty."""
    empty = Tree(None)

    def __init__(self, label, left=empty, right=empty):
        for b in (left, right):
            assert isinstance(b, BTree) or b is BTree.empty
        Tree.__init__(self, label, (left, right))

    @property
    def left(self):
        return self.branches[0]

    @property
    def right(self):
        return self.branches[1]

    def is_leaf(self):
        return [self.left, self.right] == [BTree.empty] * 2

    def __repr__(self):
        if self.is_leaf():
            return 'BTree({0})'.format(self.label)
        elif self.right is BTree.empty:
            left = repr(self.left)
            return 'BTree({0}, {1})'.format(self.label, left)
        else:
            left, right = repr(self.left), repr(self.right)
            if self.left is BTree.empty:
                left = 'BTree.empty' 
            template = 'BTree({0}, {1}, {2})'
            return template.format(self.label, left, right)
    def oth(self,other_tree):
        return self.label + other_tree.label

In [139]:
t = BTree(3,BTree(1),BTree(7,BTree(5),BTree(9,BTree.empty,BTree(11))))

In [140]:
t

BTree(3, BTree(1), BTree(7, BTree(5), BTree(9, BTree.empty, BTree(11))))

In [141]:
[1,2]*2

[1, 2, 1, 2]

In [142]:
t.left

BTree(1)

In [143]:
t.label

3

In [144]:
def fib_tree(n):
    if n==0 or n==1:
        return BTree(n)
    else:
        left = fib_tree(n-2)
        right = fib_tree(n-1)
        fib_n = left.label + right.label
        return BTree(fib_n,left,right)

In [145]:
fib_tree(3)

BTree(2, BTree(1), BTree(1, BTree(0), BTree(1)))

In [146]:
def contents(bt):
    if bt is BTree.empty:
        return []
    else:
        return contents(bt.left) + [bt.label] + contents(bt.right)

In [147]:
contents(fib_tree(5))

[1, 2, 0, 1, 1, 5, 0, 1, 1, 3, 1, 2, 0, 1, 1]

In [148]:
def balanced_bst(s):
    """Construct a binary search tree from a sorted list"""
    if not s:
        return BTree.empty
    else:
        mid = len(s)//2
        left = balanced_bst(s[:mid])
        right = balanced_bst(s[mid+1:])
        return BTree(s[mid],left,right)

In [149]:
balanced_bst(range(10))

BTree(5, BTree(2, BTree(1, BTree(0)), BTree(4, BTree(3))), BTree(8, BTree(7, BTree(6)), BTree(9)))

In [150]:
def largest(t):
    if t is BTree.empty:
        return None
    while t.right is not BTree.empty:
        t=t.right
    return t.label

In [155]:
def largest(t):
    """Return the largest element in a binary search tree.

    >>> largest(bst([1, 3, 5, 7, 9]))
    9
    """
    if t.right is BTree.empty:
        return t.label
    else:
        return largest(t.right)

def second(t):
    """Return the second largest element in a binary search tree.

    >>> second(bst([1, 3, 5]))
    3
    >>> second(bst([1, 3, 5, 7, 9]))
    7
    >>> second(Tree(1))
    """
    if t.is_leaf():
        return None
    elif t.right is BTree.empty:
        return largest(t.left)
    elif t.right.is_leaf():
        return t.label
    else:
        return second(t.right)

In [151]:
largest(balanced_bst(range(10)))

9

In [154]:
balanced_bst(range(10)).right

BTree(8, BTree(7, BTree(6)), BTree(9))

In [156]:
second(balanced_bst(range(10)))

8

In [159]:
'please vend'.strip('please ')

'vend'

In [163]:
getattr(t,'')

False

In [169]:
def f(*args):
    return args[1]

In [170]:
f(1,2,3)

2

In [171]:
t

BTree(3, BTree(1), BTree(7, BTree(5), BTree(9, BTree.empty, BTree(11))))

In [178]:
getattr(t,'oth')(t)

AttributeError: 'BTree' object has no attribute 'oth'